In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
import os

from dotenv import load_dotenv
load_dotenv()

CRYPTOCOMPARE_API_KEY = os.getenv('CRYPTOCOMPARE_API_KEY')

In [3]:
df = pd.read_parquet('coinbase.parquet', engine='pyarrow')
df['time'] = pd.to_datetime(df['time'])
df[['base_asset', 'quote_asset']] = df['product_id'].str.split('-', expand=True)
df.head()

,channel,maker_order_id,taker_order_id,time,trade_id,product_id,size,price,side,base_asset,quote_asset
0,rfq_match,5f1e264a-7760-11ef-a477-d92110f0c78d,0beb0c17-a649-4495-8256-aa327d1b6866,2024-09-20 14:55:27.044000+00:00,42539746,ETH-EUR,0.000723,2282.931149,SELL,ETH,EUR
1,rfq_match,5f217b1a-7760-11ef-b6e0-37be7b1a0976,827c80b5-28af-47f9-ad05-7028f943b00e,2024-09-20 14:55:27.093000+00:00,42539747,ALGO-USD,460.000000,0.132695,SELL,ALGO,USD
2,rfq_match,05550d9f-adb0-4f8c-bf84-ec7cd367bfe6,5fb514b8-b665-4c7b-8da7-15d4f51154f4,2024-09-20 14:55:27.138000+00:00,42539748,NEAR-USD,0.341142,4.385000,SELL,NEAR,USD
3,rfq_match,5f62a2fe-7760-11ef-b94a-97e1fbc68bb9,cdc3b641-6f63-48b9-a9a0-eb54a2e6c306,2024-09-20 14:55:27.521000+00:00,42539749,ETH-EUR,0.016277,2282.937020,SELL,ETH,EUR
4,rfq_match,f833b019-c901-4fc1-b155-b8ad8d8c7a2d,81fdac3a-f17c-49b3-9d7a-bc5084cbaa95,2024-09-20 14:55:28.473000+00:00,42539750,ALGO-USD,9.350885,0.132800,BUY,ALGO,USD


In [4]:
base_assets = df['base_asset'].unique()
quote_assets = df['quote_asset'].unique()
start_time = df['time'].min()
end_time = df['time'].max()
start_timestamp = int(start_time.timestamp())

print("Base Assets:", base_assets)
print("Quote Assets:", quote_assets)
print(f"Time Range: {start_time} to {end_time}")
print("Start Timestamp:", start_timestamp)

Base Assets: ['ETH' 'ALGO' 'NEAR' 'XRP' 'HBAR' 'VELO' 'XLM' 'COMP' 'GTC' 'ATOM' 'DOT'
 'ADA' 'SHIB' 'XTZ' 'AVAX' 'YFI' 'FLOW' 'BTC' 'KSM' 'SOL' 'ACH' 'ETC'
 'SPA' 'ANKR' 'VET' 'APT' 'MATIC' 'AMP' 'RARI' 'TIA' 'ICP' 'COTI' 'GRT'
 'AERO' 'JASMY' 'HNT' 'TRAC' 'BONK' 'GST' 'LTC' 'MANA' 'CLV' 'MPL' 'SPELL'
 'AAVE' 'QNT' 'MEDIA' 'PRO' 'SUI' 'VARA' 'MOBILE' 'BIT' 'FET' 'CRO' 'LRC'
 'DOGE' 'BCH' 'SEI' 'JTO' 'SAND' 'SUSHI' 'OP' 'MAGIC' 'ACS' 'ROSE' 'FX'
 'LDO' 'HIGH' 'SUKU' 'XYO' 'AGLD' 'ALEPH' 'DESO' 'UMA' 'IMX' 'CBETH'
 'OCEAN' 'EOS' 'FIL' 'ERN' 'FORTH' 'TVK' 'INV' 'LINK' '00' 'METIS' 'MKR'
 'KARRAT' 'QI' 'SHPING' 'APE' 'ELA' 'SWFTC' 'AIOZ' 'ABT' 'BNT']
Quote Assets: ['EUR' 'USD' 'SOL' 'USDC' 'LTC' 'GRT' 'JASMY' 'GBP' 'ETH' 'UMA' 'SHIB'
 'NEAR' 'DNT' 'BTC' 'LQTY' 'ZRX' 'ATOM' 'XRP' 'COMP' 'USDT' 'VTHO' 'BCH'
 'DOT' 'VARA' 'DOGE' 'FET' 'GTC' 'XCN' 'XTZ' 'HIGH' 'UNI' 'ICP' 'T' 'SUI'
 'MEDIA' 'SPELL' 'XLM' 'ERN' 'OP' 'LCX' 'AVAX' 'XYO' 'LINK' 'MANA' 'CRO']
Time Range: 2024-09-20 14:55:27.044000+

In [5]:
import cryptocompare
cryptocompare.cryptocompare._set_api_key_parameter(CRYPTOCOMPARE_API_KEY)

'&api_key=c574328a6b54aecfc6ff5761915d8225cd53c4355ff83a50cfc5b28206793722'

In [25]:
def compute_price(asset_ohlc):
    return (asset_ohlc['high'] + asset_ohlc['low'] + 2 * asset_ohlc['close']) / 4

asset_prices = {}

for asset in quote_assets:
    raw_price = cryptocompare.get_historical_price_day(asset, 'USD', limit=1, toTs=start_timestamp)
    if not raw_price:
        print('no data for {asset}')
        asset_prices[asset] = np.float64(0.0)
    else:
        asset_ohlc = pd.DataFrame.from_dict(raw_price).tail(1)
        asset_price = compute_price(asset_ohlc.iloc[0])
        
        asset_prices[asset] = asset_price

[ERROR] CCCAGG market does not exist for this coin pair (ERN-USD)
no data for {asset}


In [27]:
grouped = df.groupby([pd.Grouper(key='time', freq='min'), 'product_id', 'quote_asset']).agg(
    total_quantity=('size', 'sum'),
    avg_quote_price=('price', 'mean'),
    total_trades=('trade_id', 'count')
).reset_index()

In [28]:
grouped_sorted = grouped.sort_values(['time', 'total_trades'], ascending=[True, False])
grouped_top5 = grouped_sorted.groupby('time').head(5)

fig_trades = px.bar(grouped_top5, x='time', y='total_trades', color='product_id', title='Trades by product over time (1-minute)')
fig_trades.update_layout(xaxis_title='time', yaxis_title='#trades')
fig_trades.show()

In [38]:
priced = grouped.copy()

priced['quote_asset_usd'] = priced['quote_asset'].map(asset_prices)
priced['total_usd'] = priced['total_quantity'] * priced['avg_quote_price'] * priced['quote_asset_usd']

priced.head()

,time,product_id,quote_asset,total_quantity,avg_quote_price,total_trades,quote_asset_usd,total_usd
0,2024-09-20 14:55:00+00:00,ACH-BTC,BTC,5282.044158,3.123261e-07,1,63224.932500,104.303458
1,2024-09-20 14:55:00+00:00,ADA-USD,USD,71.872956,3.554932e-01,2,1.000325,25.558653
2,2024-09-20 14:55:00+00:00,ALGO-ETH,ETH,26.099633,5.220652e-05,2,2533.630000,3.452251
3,2024-09-20 14:55:00+00:00,ALGO-EUR,EUR,92.472962,1.190300e-01,1,1.116500,12.289383
4,2024-09-20 14:55:00+00:00,ALGO-JASMY,JASMY,73.240071,6.333806e+00,1,0.021140,9.806601


In [49]:
grouped_sorted = priced.sort_values(['time', 'total_usd'], ascending=[True, False])
grouped_top5 = grouped_sorted.groupby('time').head(5)

fig_trades = px.bar(grouped_top5, 
                    x='time', y='total_usd', color='product_id', 
                    title='Volume by product over time (1-minute)',
                    hover_data={'total_usd': ':$.2f'})
fig_trades.update_layout(xaxis_title='time', yaxis_title='USD volume')
fig_trades.show()